## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
#LWM을 하기위한 라이브러리 가져오기
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import time


plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

## GPU설정

In [2]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# DeepMIMOv3 다운

In [3]:
# pip install DeepMIMOv3 umap-learn

## 파라미터 수정

In [4]:
## Load and print the default parameters
# bandwith: 0.05GHz(50MHz 대역폭 사용)
parameters = DeepMIMOv3.default_params()
pprint(parameters)

{'OFDM': {'RX_filter': 0,
          'bandwidth': 0.05,
          'selected_subcarriers': array([0]),
          'subcarriers': 512},
 'OFDM_channels': 1,
 'active_BS': array([1]),
 'bs_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([8, 4]),
                'spacing': 0.5},
 'dataset_folder': './Raytracing_scenarios',
 'dynamic_scenario_scenes': array([1]),
 'enable_BS2BS': 1,
 'enable_doppler': 0,
 'enable_dual_polar': 0,
 'num_paths': 5,
 'scenario': 'O1_60',
 'ue_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([4, 2]),
                'spacing': 0.5},
 'user_rows': array([1]),
 'user_subsampling': 1}


In [5]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM 동적 시나리오 불러오기
#자신의 LWM 파일 위치 경로 작성
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
scene = 10 # 장면 수
parameters['dataset_folder'] = r'C:\Users\dlghd\졸업프로젝트\LWM'

# scnario = 02_dyn_3p5 <- 다운받은 파일(동적시나리오)
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(scene) #scene 0~9

# 각 사용자-기지국 채널에 대해 최대 10개 멀티패스 경로 사용
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [6]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 1, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable_doppler': 0, 'enable_dual_polar'

## dataset 구축

In [7]:
## dataset 구축 (chunked on‑the‑fly generation)
import time, gc
from tqdm import tqdm

# 0~999 씬 인덱스, 한 번에 50개씩 처리
scene_indices = np.arange(scene)
chunk_size   = 5
all_data     = []

# 씬 묶음(chunk)마다 generate_data 호출
for i in tqdm(range(0, len(scene_indices), chunk_size)):
    chunk = scene_indices[i : i+chunk_size].tolist()
    parameters['dynamic_scenario_scenes'] = chunk

    start = time.time()
    data_chunk = DeepMIMOv3.generate_data(parameters)
    print(f"Scenes {chunk[0]}–{chunk[-1]} generation time: {time.time() - start:.2f}s")

    # 바로 all_data에 합치거나, 디스크에 저장해도 OK
    all_data.extend(data_chunk)

    # 메모리 해제
    del data_chunk
    gc.collect()

# 마지막에 하나의 리스트로 합친 데이터셋
dataset = all_data


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|████████████████████████████████████████████████████| 69040/69040 [00:02<00:00, 32604.65it/s]

Generating channels:  36%|██████████████████▉                                  | 24731/69040 [00:05<00:08, 5164.42it/s]


Generating channels:  83%|███████████████████████████████████████████▉         | 57255/69040 [00:13<00:02, 4027.51it/s]


Generating channels: 100%|█████████████████████████████████████████████████████| 69040/69040 [00:13<00:00, 5086.33it/s]


...

Generating channels:  76%|████████████████████████████████████████             | 52211/69040 [00:12<00:04, 3892.49it/s]


Generating channels: 100%|█████████████████████████████████████████████████████| 69040/69040 [00:13<00:00, 5030.10it/s]



BS-BS Channels



Reading ray-tracing: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:46<00:00, 83.22s/it]

Scenes 5–9 generation time: 82.95s


# 사용자 접근 데이터

In [8]:
user_data = dataset[0][0]['user']
print(user_data.keys())

dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# 사용자 채널 정보 확인

In [9]:
# subcarries = 나눈 각각의 주파수 채널
# Channel = H <- 채널 벡터
# 채널 형태
# (user, UE antenna, Bs antenna, subcarrier)
channel = dataset[0][0]['user']['channel']
print(channel.shape)  

(69040, 1, 32, 64)


In [10]:
print(dataset[0][0]['user']['channel'][100])

[[[-4.9276509e-06+6.0179661e-07j -4.7681883e-06+1.3829425e-06j
   -4.4857170e-06+2.1285541e-06j ...  4.3711116e-06-2.4074948e-06j
    3.9297033e-06-3.0764131e-06j  3.3868639e-06-3.6660977e-06j]
  [-4.7825752e-06+1.6998159e-06j -4.4491662e-06+2.4436672e-06j
   -4.0009481e-06+3.1246111e-06j ...  3.8229477e-06-3.3767817e-06j
    3.2333687e-06-3.9455144e-06j  2.5602983e-06-4.4125736e-06j]
  [-4.3861578e-06+2.7614337e-06j -3.8878534e-06+3.4282084e-06j
   -3.2891933e-06+4.0066625e-06j ...  3.0543217e-06-4.2173128e-06j
    2.3400164e-06-4.6522073e-06j  1.5652473e-06-4.9671735e-06j]
  ...
  [-5.9705985e-06+1.8381670e-06j -5.5992200e-06+2.7702260e-06j
   -5.0834296e-06+3.6307945e-06j ...  4.8462462e-06-3.9304277e-06j
    4.1544481e-06-4.6554678e-06j  3.3555179e-06-5.2603964e-06j]
  [-5.3850690e-06+3.0986103e-06j -4.8194606e-06+3.9206407e-06j
   -4.1295657e-06+4.6415066e-06j ...  3.8328362e-06-4.8787911e-06j
    3.0023016e-06-5.4293314e-06j  2.0943648e-06-5.8398036e-06j]
  [-4.5357333e-06+4.1878

In [11]:
print(len(dataset[0][0]['user']['channel'][100]))

1


In [12]:
print(channel[10000][0][0])

[ 1.14151417e-05-4.36109121e-06j  1.13104134e-05-4.99916769e-06j
  1.11716581e-05-5.67118514e-06j  1.09873472e-05-6.37486755e-06j
  1.07464039e-05-7.10507175e-06j  1.04388646e-05-7.85394786e-06j
  1.00564839e-05-8.61123954e-06j  9.59326280e-06-9.36472679e-06j
  9.04586159e-06-1.01007672e-05j  8.41387282e-06-1.08049298e-05j
  7.69995040e-06-1.14626673e-05j  6.90977913e-06-1.20600052e-05j
  6.05189052e-06-1.25842089e-05j  5.13732675e-06-1.30243880e-05j
  4.17918318e-06-1.33720132e-05j  3.19203900e-06-1.36213148e-05j
  2.19131471e-06-1.37695433e-05j  1.19258925e-06-1.38170753e-05j
  2.10908027e-07-1.37673669e-05j -7.39877976e-07-1.36267463e-05j
 -1.64770893e-06-1.34040656e-05j -2.50281732e-06-1.31102279e-05j
 -3.29810814e-06-1.27576113e-05j -4.02939668e-06-1.23594264e-05j
 -4.69549514e-06-1.19290335e-05j -5.29813951e-06-1.14792647e-05j
 -5.84176632e-06-1.10217779e-05j -6.33314448e-06-1.05664840e-05j
 -6.78089191e-06-1.01210744e-05j -7.19489299e-06-9.69066787e-06j
 -7.58565420e-06-9.277610

In [13]:
print(channel[1][0][0])

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


# 사용자 위치 정보

In [14]:
location = dataset[0][0]['user']['location']
print(location.shape)      # (사용자 수, 3)
print(location[0:4])         # 첫 번째 사용자의 (x, y, z)

(69040, 3)
[[-91.03330231 -15.57629967   1.        ]
 [-90.83329773 -15.57629967   1.        ]
 [-90.63330078 -15.57629967   1.        ]
 [-90.4332962  -15.57629967   1.        ]]


# 경로정보

In [15]:
paths = dataset[0][0]['user']['paths']
#사용자 수
print(len(paths))
# 첫 번째 사용자 경로 정보
print(paths[0])

69040
{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []}


# 기지국 정보

In [16]:
bs_data = dataset[0][0]['basestation']
print(bs_data.keys())


dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# Scene 및 사용자 수

In [22]:
for i, scene in enumerate(dataset[0]):
    user_locs = scene['user']['location']
    print(f"Scene {i}: {len(user_locs)} users")

Scene 0: 69040 users


# 채널 수

In [23]:
len(dataset[0][0]['user']['channel'])

69040

In [24]:
print(dataset[0][0]['user']['paths'][0])

{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []}


In [25]:
scene = dataset[0][0] # scene 0
ue_idx = 0 # 첫 번째 사용자
channel = scene['user']['channel'][ue_idx]
print(channel.shape)

(1, 32, 64)


# channel CIR mat 정보 가져오기

In [26]:
import scipy.io as sio

file_path = r'C:\Users\dlghd\졸업프로젝트\LWM\O2_dyn_3p5\scene_0\O2_dyn_3p5.1.CIR.mat'
mat_data = sio.loadmat(file_path)

# 파일 안의 key 확인
print(mat_data.keys())




dict_keys(['__header__', '__version__', '__globals__', 'CIR_array_full'])


In [27]:
# 일반적으로 CIR key는 'CIR' 또는 'cir' 같은 이름일 가능성 높음
H_cir = mat_data['__header__']  
print(H_cir)

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Jun 30 11:33:01 2021'


# Time-Prediction 시작
## Time Series 형태로 변환
### 단일사용자 채널 예측

In [28]:
# print(dataset[0][0]['user']['channel'][150][0][3])

count = 0
for h in dataset[0][0]['user']['channel'][100][0]:
#     h = h.squeeze(0)
    h_real = h.real
    h_imag = h.imag
    if np.sum(np.abs(h_real)) ==0:
        count+=1
    elif np.sum(np.abs(h_imag)) == 0:
        count+=1

print("0이 존재하는 채널 개수",count)

0이 존재하는 채널 개수 0


In [29]:
import numpy as np

# 1) (user, ue_port, bs_ant, subc) → (bs_ant, subc) 로 squeeze
H = dataset[0][0]['user']['channel'][100, 0]   # shape: (32, 64), complex

# 2) BS 안테나 인덱스 3의 서브캐리어 벡터 (64,)
print("Antenna #3 subcarriers:", H[3])

# 3) 전체 서브캐리어(32×64) 중 값이 정확히 0인 요소 개수
zero_elements = np.sum(H == 0)
print("0+0j인 서브캐리어 개수:", zero_elements)

# 4) 서브캐리어 전부가 0인 안테나 포트(행) 개수
zero_ports = np.sum(np.all(H == 0, axis=1))
print("완전 0+0j 안테나 포트 개수:", zero_ports)

# 5) 만약 “값이 하나도 0이 아닌” 서브캐리어 요소 개수를 보고 싶다면
nonzero_elements = np.sum(np.abs(H) > 0)
print("0이 아닌 서브캐리어 개수:", nonzero_elements)


Antenna #3 subcarriers: [-3.7481711e-06+3.7276918e-06j -3.1033380e-06+4.2799297e-06j
 -2.3783671e-06+4.7218546e-06j -1.5919456e-06+5.0420513e-06j
 -7.6434952e-07+5.2322434e-06j  8.3081005e-08+5.2875071e-06j
  9.2849081e-07+5.2063970e-06j  1.7500737e-06+4.9909859e-06j
  2.5266352e-06+4.6468099e-06j  3.2381383e-06+4.1827284e-06j
  3.8662224e-06+3.6106942e-06j  4.3946743e-06+2.9454461e-06j
  4.8098500e-06+2.2041299e-06j  5.1010238e-06+1.4058554e-06j
  5.2606679e-06+5.7120445e-07j  5.2846453e-06-2.7829972e-07j
  5.1723187e-06-1.1207479e-06j  4.9265650e-06-1.9344095e-06j
  4.5537045e-06-2.6982937e-06j  4.0633354e-06-3.3926899e-06j
  3.4680893e-06-3.9996776e-06j  2.7833046e-06-4.5035881e-06j
  2.0266314e-06-4.8914089e-06j  1.2175759e-06-5.1531201e-06j
  3.7699922e-07-5.2819537e-06j -4.7342218e-07-5.2745672e-06j
 -1.3117545e-06-5.1311317e-06j -2.1163728e-06-4.8553270e-06j
 -2.8665186e-06-4.4542485e-06j -3.5428352e-06-3.9382230e-06j
 -4.1278677e-06-3.3205442e-06j -4.6065129e-06-2.6171292e-06j


In [30]:
# 결측치 제거 예시 코드
import numpy as np
print(np.real_if_close(sum(dataset[0][0]['user']['channel'][1][0][0])))


0.0


In [31]:
ex = dataset[0][0]['user']['channel'].squeeze(1)

In [32]:
ex.shape

(69040, 32, 64)

In [33]:
ex[10][31][63]

0j

In [34]:
ex1 = np.nan_to_num(ex, nan=0.0)
print(ex1[0])

[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]


## 결측치 제거 및 dataload

In [50]:
from torch.utils.data import Dataset, DataLoader

class ChannelDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        scene = self.dataset[idx]
        H = scene[0]['user']['channel'].squeeze(1)
        
        mask_valid_sc = ~np.all(H == 0+0j, axis=(0,1))
        
        H = H[:,:,mask_valid_sc]
        
        H_real = H.real
        H_imag = H.imag
        H_concat  = np.concatenate([H_real, H_imag], 1)
        H_sc_first = H_concat.transpose(0,2,1)
        
        return torch.from_numpy(H_sc_first.astype(np.float32))   
            
            

In [51]:
dataset[0]

[{'user': {'paths': array([{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          ...,
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
          {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []}],
         dtype=object),
   'LoS': array([-1, -1, -1, ..., -1, -1, -1], dtype=int8),
   'location': array([[-91.03330231, -15.57

In [52]:
print(dataset[0][0]['user']['channel'])

[[[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 ...


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j

In [53]:
# ─────────────────────────────────────────────
# ❶ IterableDataset: 모든 유저·서브캐리어를 스트리밍
import torch
from torch.utils.data import IterableDataset, DataLoader

class ChannelSeqDataset(IterableDataset):
    def __init__(self, scenes, seq_len=5):
        super().__init__()
        self.scenes   = scenes
        self.seq_len  = seq_len
        ch0           = scenes[0][0]['user']['channel']
        self.U        = ch0.shape[0]      # 유저 수
        self.A        = ch0.shape[2]      # 안테나 32
        self.S        = ch0.shape[3]      # 서브캐리어 64
    def _vec(self, scene, u, sc):
        h = scene[0]['user']['channel'][u,0,:,sc]
        return torch.from_numpy(
            np.concatenate([h.real, h.imag]).astype(np.float32)
        )                                   # (2*A,)
    def __iter__(self):
        T = len(self.scenes)
        for t in range(self.seq_len, T):
            past = self.scenes[t-self.seq_len:t]
            tgt  = self.scenes[t]
            for u in range(self.U):
                for s in range(self.S):
                    seq = torch.stack([self._vec(ps,u,s) for ps in past])
                    if not torch.any(seq):        # 전부 0 → skip
                        continue
                    target      = self._vec(tgt,u,s)
                    masked_pos  = torch.tensor([self.seq_len-2])
                    yield seq, masked_pos, target
# ─────────────────────────────────────────────
# ❷ 학습·검증 DataLoader
seq_len       = 5
split_ratio   = 0.8
split_idx     = int(len(dataset)*split_ratio)

train_ds      = ChannelSeqDataset(dataset[:split_idx], seq_len=seq_len)
val_ds        = ChannelSeqDataset(dataset[split_idx:], seq_len=seq_len)

batch_size    = 32
train_loader  = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
val_loader    = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
# ─────────────────────────────────────────────


In [54]:
# print(X.shape)
# print(y.shape)

In [55]:
# # 데이터 만들기
# # seq_len = 시퀀스를 잘라서 사용
# # [0,1,2,3,4], [1,2,3,4,5], [2,3,4,5]
# seq_len = 5
# X = []
# y = []

# for i in range(len(channel_sequence) - seq_len+1):
#     input_seq = channel_sequence[i:i+seq_len-1] # t=0~3
#     target = channel_sequence[i+seq_len-1] # t = 4
    
#     X.append(input_seq)
#     y.append(target)

# # T = scene case
# # samples = T - seq_len + 1
# X = np.array(X) # shape: (samples, seq_len-1, 64, 64)
# y = np.array(y) # shape: (samples, 64, 64)


In [56]:
# # Flatten
# # 4096 = 64 * 64
# X_flat = X.reshape(X.shape[0], X.shape[1], -1) # (320,5,64)
# y_flat = y.reshape(y.shape[0],-1) #(\

# # power = np.sum(y_flat**2,axis=1)
# # mask = power > 0
# # y_flat = y_flat[mask]
# # X_flat = X_flat[mask]

In [57]:
# print(X_flat.shape)
# # print(y_flat.shape)

In [58]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader, random_split

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # 아래 텐서 cpu로
# X_t = torch.tensor(X_flat, dtype=torch.float32)        # (N, seq_len, element_length)
# y_t = torch.tensor(y_flat, dtype=torch.float32)        # (N,)
# masked_pos = torch.full((X_t.shape[0], 1),             # (N, 1)
#                         seq_len - 2,
#                         dtype=torch.long)

# #  Dataset 생성
# full_ds = TensorDataset(X_t, masked_pos, y_t)

# # train / val 8:2 분할
# train_size = int(len(full_ds) * 0.8)
# val_size   = len(full_ds) - train_size
# train_ds, val_ds = random_split(full_ds, [train_size, val_size])

# # DataLoader 정의
# batch_size = 32

# train_loader = DataLoader(
#     train_ds,
#     batch_size=batch_size,
#     shuffle=True,        # epoch마다 섞기
#     drop_last=False
# )

# val_loader = DataLoader(
#     val_ds,
#     batch_size=batch_size,
#     shuffle=False
# )


In [59]:
# print(masked_pos.shape)

In [60]:
# print(X_t)

In [61]:
# print(X_t[3][0][1])

In [62]:
# print(X_t.shape)
# print(y_t.shape)
# # mini_batch, sequence_len, H_real+H_imag

# 아래 코드 구조
┌──────────────────────────────────────────────────────────────┐
│ input_ids  (B, seq_len, element_length)  ─┐                 │
│ masked_pos (B, num_mask)                  ├─>  LWM backbone │
│                                           │    (12-층 트랜스포머)  
└────────────────────────────────────────────┘         │
            logits_lm  (B, num_mask, element_length)  │   enc_output (B, seq_len, d_model)
                                                      ▼
                        ┌─[풀링]───────────────┐      ←── feat (B, d_model)
                        │ 첫 토큰(0번) 선택    │
                        │   or 평균/최대 풀링 │
                        └──────────────────────┘
                                      ▼
                       FC 헤드  (d_model → hidden_dim → out_dim)
                                      ▼
                                out (B, out_dim)

# 시각적비유

[패치 프로젝터]──▶[Transformer ×12]──▶[LayerNorm]──┐
                                                  ├─▶ 64-차 벡터 (CLS 또는 풀링) ─▶ MLP ─▶ out                                                
[Positional Embedding]─────────────────────────────┘


In [63]:
"""
LWMWithHead: 사전학습된 LWM(Transformer encoder)을 ‘백본(backbone)’으로 사용하고,
그 뒤에 새로운 완전연결(FC) 헤드(head)를 붙여 다운스트림 작업(회귀·분류 등)에
사용할 수 있도록 만든 래퍼(wrapper) 클래스입니다.
"""

import torch
import torch.nn as nn
from lwm_model import lwm   # 기존 LWM 모델 클래스 (import 경로는 프로젝트 구조에 맞게 조정)

class LWMWithHead(nn.Module):
    """
    Args
    ----
    element_length : int
        LWM 입력 패치의 길이. 예) 64*64 = 4096 (H_real + H_imag)
    d_model        : int
        Transformer 모델 차원(=LWM hidden size).
    max_len        : int
        포지셔널 임베딩 최대 길이(시퀀스 길이).
    n_layers       : int
        Transformer 인코더 층 수.
    hidden_dim     : int
        새 FC 헤드의 중간 차원.
    out_dim        : int
        최종 출력 차원. 1 → 회귀/이진분류, k → k-클래스 분류.
    freeze_backbone: bool
        True면 백본을 동결(freeze)하여 헤드만 학습.
    ckpt_path      : str | None
        사전학습 가중치(.pth) 경로. None이면 랜덤 초기화.
    device         : str
        'cuda' / 'cpu' 등 모델을 올릴 장치.
    """

    def __init__(
        self,
        element_length: int,
        d_model: int = 64,
        max_len: int = 129,
        n_layers: int = 12,
        hidden_dim: int = 256,
        out_dim: int = 64, 
        freeze_backbone: bool = False,
        ckpt_path: str | None = None,
        device: str = "cuda",
    ):
        super().__init__()

        # ────────────────────────────
        # 1) 백본(backbone) 초기화
        # ────────────────────────────
        if ckpt_path is None:
            # 가중치 없이 새로 생성
            self.backbone = lwm(
                element_length=element_length,
                d_model=d_model,
                max_len=max_len,
                n_layers=n_layers
            ).to(device)
        else:
            # 사전학습 가중치 로드
            self.backbone = lwm.from_pretrained(
                ckpt_name=ckpt_path,
                device=device
            )

        # 백본 동결(선택)
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # ────────────────────────────
        # 2) 헤드(head) 정의
        # ────────────────────────────
        self.head = nn.Sequential(
            nn.Linear(d_model, 64),  # 첫 FC
            nn.ReLU(),                       # 활성화
            nn.Linear(64, out_dim)   # 최종 FC
        )

    # ────────────────────────────
    # forward
    # ────────────────────────────
    def forward(self, input_ids, masked_pos):
        """
        Parameters
        ----------
        input_ids : Tensor  (B, seq_len, element_length)
            LWM 입력 시퀀스 (패치/토큰 단위 실수·복소수 채널값 등).
        masked_pos : Tensor  (B, num_mask)
            LWM의 마스크드 채널 모델링용 인덱스 (백본 규격 유지용).

        Returns
        -------
        out : Tensor  (B, out_dim)
            헤드에서 계산된 다운스트림 작업용 로짓/예측값.
        """

        # 기존 LWM forward:
        #   logits_lm : (B, num_mask, element_length)  ← 사용 안 함
        #   enc_output: (B, seq_len, d_model)
        _, enc_output = self.backbone(input_ids, masked_pos)

        # 특징 추출(feat)
        # ① 첫 토큰 벡터 사용 (CLS 개념) ─────────────
        feat = enc_output[:, 0, :]           # (B, d_model)

        # ② 평균 풀링 예시 (필요 시 주석 해제) ─────
        # feat = enc_output.mean(dim=1)       # (B, d_model)

        # ③ Max 풀링 예시 (필요 시 주석 해제) ─────
        # feat, _ = enc_output.max(dim=1)     # (B, d_model)

        # 헤드 통과 → 최종 출력
        out = self.head(feat)                # (B, out_dim)
        return out


In [70]:
print(train_ds.U)

69040


In [65]:
import torch
import torch.nn as nn
from torch.optim import Adam

# 디바이스 설정(GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# 모델 초기화
seq_len = 5
element_length=train_ds.A*2



model = LWMWithHead(
    element_length=element_length,  # 예: 64
    d_model=64,
    max_len=seq_len,                # 예: 5
    n_layers=12,
    hidden_dim=256,
    out_dim=element_length,         # 예: 64
    freeze_backbone=False,  
    ckpt_path=None,  
    device=device
).to(device)

# 손실함수
criterion = nn.MSELoss()

# 옵티마이저 설정
optimizer = Adam(model.parameters(), lr=1e-4)

Using device: cuda


In [71]:
# import time
# import sys

# #  ❶ quick sanity checks
# #    If train_loader is empty, this will print 0 or raise
# try:
#     n_batches = len(train_loader)
# except TypeError:
#     # IterableDataset → no __len__
#     n_batches = sum(1 for _ in train_loader)
# print(f"→ training batches: {n_batches}")

# # ❷ actual training loop with per-epoch print
# start = time.time()
# num_epochs = 10

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0

#     # loop over batches
#     for i, (input_ids, masked_pos, target) in enumerate(train_loader, 1):
#         input_ids   = input_ids.to(device)
#         masked_pos  = masked_pos.to(device)
#         target      = target.to(device)

#         optimizer.zero_grad()
#         pred = model(input_ids, masked_pos).squeeze(-1)
#         loss = criterion(pred, target)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     # ❸ now we know at least one epoch happened
#     avg_loss = running_loss / i if i>0 else float('nan')
#     print(f"Epoch {epoch+1}/{num_epochs},  avg loss: {avg_loss:.6f}", flush=True)

# end = time.time()
# print(f"Total training time: {end - start:.2f}s")


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
import time

num_epochs = 10
start = time.time()
for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0.0

    # tqdm 으로 배치 단위 진행바 감싸기
    for batch_idx, (input_ids, masked_pos, target) in enumerate(
            tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"), 1):

        input_ids, masked_pos, target = (
            input_ids.to(device),
            masked_pos.to(device),
            target.to(device),
        )
        optimizer.zero_grad()
        pred = model(input_ids, masked_pos).squeeze(-1)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # (선택) 100 배치마다 로그
        if batch_idx % 100 == 0:
            tqdm.write(f"  [Batch {batch_idx}]  loss: {running_loss/batch_idx:.6f}")

    avg_loss = running_loss / batch_idx
    print(f"→ Epoch {epoch}/{num_epochs} done, avg_loss: {avg_loss:.6f}")

print(f"Total training time: {time.time() - start:.2f}s")


Epoch 1/10: 102it [00:04, 23.50it/s]

  [Batch 100]  loss: 0.000001


Epoch 1/10: 204it [00:08, 23.10it/s]

  [Batch 200]  loss: 0.000001


Epoch 1/10: 303it [00:13, 22.44it/s]

...

Epoch 1/10: 5102it [03:59, 22.04it/s]

  [Batch 5100]  loss: 0.000001


Epoch 1/10: 5204it [04:04, 22.16it/s]

  [Batch 5200]  loss: 0.000001


Epoch 1/10: 5291it [04:08, 22.20it/s]

In [68]:
print(input_ids.shape)
print(masked_pos.shape)
print(target.shape)
print(pred.shape)

torch.Size([32, 5, 64])
torch.Size([32, 1])
torch.Size([32, 64])
torch.Size([32, 64])


In [ ]:
# 모델 평가 방법
import torch.nn.functional as F

def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    RMSE = Root MSE
    RMSE = {1/n*sum((y^-y)**2)}**1/2
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
    returns: 스칼라 (배치 평균)
    """
    # (B, …) → (B,)  : 각 샘플별 제곱합
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1)
    return (mse_per_sample / power_per_sample).mean()


In [ ]:
# 모델 평가 함수
import torch
import torch.nn.functional as F

# ─────────────────────────────────────────
# 1. 배치 단위 RMSE, NMSE 함수
# ─────────────────────────────────────────
def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Root-Mean-Squared Error
    returns: 스칼라 (배치 평균)
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
      · 채널 예측 분야에서 흔히 쓰는 지표
    returns: 스칼라 (배치 평균)
    """
    # (B, …) → (B,)  : 각 샘플별 제곱합
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1)
    return (mse_per_sample / power_per_sample).mean()


# ─────────────────────────────────────────
# 2. 검증 루프 예시
# ─────────────────────────────────────────
def evaluate(model, loader, device="cuda"):
    model.eval()
    total_rmse, total_nmse = 0.0, 0.0

    with torch.no_grad():
        for input_ids, masked_pos, target in loader:
            input_ids  = input_ids.to(device)
            masked_pos = masked_pos.to(device)
            target     = target.to(device)

            pred = model(input_ids, masked_pos)

            total_rmse += rmse(pred, target).item() * input_ids.size(0)
            total_nmse += nmse(pred, target).item() * input_ids.size(0)

    N = len(loader.dataset)
    return {
        "RMSE": total_rmse / N,
        "NMSE": total_nmse / N
    }


In [ ]:
num_epochs = 30
patience_counter = 0

for epoch in range(1, num_epochs + 1):
    # ──────────── ① 학습 ────────────
    model.train()
    for input_ids, masked_pos, target in train_loader:
        input_ids  = input_ids.to(device)
        masked_pos = masked_pos.to(device)
        target     = target.to(device)

        pred = model(input_ids, masked_pos)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # ──────────── ② 검증 ────────────
    metrics = evaluate(model, val_loader, device)
    rmse_val, nmse_val = metrics["RMSE"], metrics["NMSE"]

    # ──────────── ③ 로그 출력 ────────────
    print(f"[Epoch {epoch:02d}]  "
          f"val_RMSE = {rmse_val:.4f}   "
          f"val_NMSE = {nmse_val:.4e}   "
          f"val_NMSE(dB) = {10*torch.log10(torch.tensor(nmse_val)):.2f} dB")
